In [1]:
import pandas as pd

In [3]:
# World Development Indicators
wdi = pd.read_parquet('indicators.parquet', engine='fastparquet')

# Happiness datasets for 2015–2019
h15 = pd.read_parquet('2015.parquet')
h16 = pd.read_parquet('2016.parquet')
h17 = pd.read_parquet('2017.parquet')
h18 = pd.read_parquet('2018.parquet')
h19 = pd.read_parquet('2019.parquet')

In [4]:
# Melt all year columns into rows
wdi_long = wdi.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="year",
    value_name="indicator_value"
)

In [6]:
# Convert year to integer
wdi_long["year"] = wdi_long["year"].astype(int)

# Filter to 2015–2019 only
wdi_long = wdi_long.query("2015 <= year <= 2019")

In [8]:
# Turn indicators into columns

wdi_wide = wdi_long.pivot_table(
    index=["Country Name", "year"],
    columns="Indicator Name",
    values="indicator_value"
).reset_index()

In [22]:
wdi_wide.rename(columns={"Country Name": "country"}, inplace=True)

At This Point, WDI is in the proper format: Country, Year, Indicator 1, ... Indicator n

In [23]:
# Show only the first 8 columns for readability
cols = wdi_wide.columns[:8]  # country, year, plus first 6 indicators
wdi_wide[cols].head(11)

Indicator Name,country,year,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)"
0,Afghanistan,2015,61.5,27.600000,11.400000,79.500000,71.500000,64.600000
1,Afghanistan,2016,NaN,28.800000,12.600000,80.500000,97.700000,97.100000
2,Afghanistan,2017,NaN,30.300000,13.500000,81.600000,97.700000,97.100000
3,Afghanistan,2018,67.7,31.400000,14.500000,82.600000,93.400000,91.600000
4,Afghanistan,2019,NaN,32.600000,15.600000,83.200000,97.700000,97.100000
5,Africa Eastern and Southern,2015,NaN,18.001597,7.096003,38.488233,33.922276,16.527554
6,Africa Eastern and Southern,2016,NaN,18.558234,7.406706,38.779953,38.859598,24.627753
7,Africa Eastern and Southern,2017,NaN,19.043572,7.666648,39.068462,40.223744,25.432092
8,Africa Eastern and Southern,2018,NaN,19.586457,8.020952,39.445526,43.035073,27.061929
9,Africa Eastern and Southern,2019,NaN,20.192064,8.403358,39.818645,44.390861,29.154282


In [24]:
print(wdi_wide.columns)

Index(['country', 'year',
       'ARI treatment (% of children under 5 taken to a health provider)',
       'Access to clean fuels and technologies for cooking (% of population)',
       'Access to clean fuels and technologies for cooking, rural (% of rural population)',
       'Access to clean fuels and technologies for cooking, urban (% of urban population)',
       'Access to electricity (% of population)',
       'Access to electricity, rural (% of rural population)',
       'Access to electricity, urban (% of urban population)',
       'Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)',
       ...
       'Women who believe a husband is justified in beating his wife (any of five reasons) (%)',
       'Women who believe a husband is justified in beating his wife when she argues with him (%)',
       'Women who believe a husband is justified in beating his wife when she burns the food (%)',
       'Women who believe a hus

Adding in Lables (Happiness Scores)

In [30]:
def clean_happiness(df, year):
    df = df.rename(columns={
        "Country or region": "country",
        "Country": "country",
        "Happiness.Score": "score",
        "Happiness Score": "score",
        "Score": "score",
    })
    df["year"] = year
    return df[["country", "score", "year"]]

In [31]:
h15 = clean_happiness(h15, 2015)
h16 = clean_happiness(h16, 2016)
h17 = clean_happiness(h17, 2017)
h18 = clean_happiness(h18, 2018)
h19 = clean_happiness(h19, 2019)

In [32]:
happiness = pd.concat([h15, h16, h17, h18, h19], ignore_index=True)

In [35]:
merged = happiness.merge(
    wdi_wide,
    left_on=["country", "year"],
    right_on=["country", "year"],
    how="inner"
)

In [36]:
merged.to_parquet("dataset.parquet", index=False)

At this point the dataset has the correct structure

In [ ]:
df = pd.read_parquet("dataset.parquet")

,country,score,year,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
0,Switzerland,7.587,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.437007,100.0
1,Iceland,7.561,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.673923,100.0
2,Denmark,7.527,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.312663,100.0
3,Norway,7.522,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.514336,NaN
4,Canada,7.427,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.485395,200.0


In [45]:
df.head()

,country,score,year,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
0,Switzerland,7.587,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.437007,100.0
1,Iceland,7.561,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.673923,100.0
2,Denmark,7.527,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.312663,100.0
3,Norway,7.522,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.514336,NaN
4,Canada,7.427,2015,NaN,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.485395,200.0


In [44]:
print(df[df["country"] == "United States"].to_string())

           country  score  year  ARI treatment (% of children under 5 taken to a health provider)  Access to clean fuels and technologies for cooking (% of population)  Access to clean fuels and technologies for cooking, rural (% of rural population)  Access to clean fuels and technologies for cooking, urban (% of urban population)  Access to electricity (% of population)  Access to electricity, rural (% of rural population)  Access to electricity, urban (% of urban population)  Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)  Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)  Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)  Account ownership at a financial institution or with a mobile-money-service provider, older adults (% of population ages 25+)  Account ownership at a financial ins

Now checking usability of different indicators

In [49]:
# fraction of missing values per column
missing_fraction = df.isna().mean()

# count how many have 0 missingness
num_complete_columns = (missing_fraction < 0.2).sum()

print("Number of columns with < 20% missingness:", num_complete_columns)

Number of columns with < 20% missingness: 725


In [50]:
usable_cols = missing_fraction[missing_fraction < 0.20].index.tolist()

In [51]:
df_usable = df[usable_cols]

In [57]:
df_usable.head()

,score,year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),...,Voice and Accountability: Standard Error,"Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100),Women's share of population ages 15+ living with HIV (%)
0,7.587,2015,100.0,100.0,100.0,100.0,100.0,100.0,3.829646,5.495156e+11,...,0.152728,10.463604,10.099898,10.268806,86.896059,83.402880,85.024916,406.888604,85.625,26.437007
1,7.561,2015,100.0,100.0,100.0,100.0,100.0,100.0,8.662795,1.398266e+10,...,0.152728,7.496144,16.520870,12.241124,91.548103,81.380837,86.202403,62.673383,96.875,28.673923
2,7.527,2015,100.0,100.0,100.0,100.0,100.0,100.0,2.835303,2.602923e+11,...,0.144910,4.377631,7.459880,6.019697,94.200167,88.443370,91.133237,371.715844,100.000,26.312663
3,7.522,2015,100.0,100.0,100.0,100.0,100.0,100.0,-0.748130,3.177570e+11,...,0.142751,4.330660,8.057614,6.281178,95.216584,90.740970,92.874207,143.056507,96.875,33.514336
4,7.427,2015,100.0,100.0,100.0,100.0,100.0,100.0,-2.396463,1.262658e+12,...,0.144154,8.982606,11.808995,10.474294,88.454140,81.910413,85.000559,43.563079,97.500,23.485395


In [ ]:
# fill missing values with median per country
df_usable = df_usable.groupby("country").transform(lambda x: x.fillna(x.median()))

c:\Users\bella\Documents\GitHub\CountryHappinessPredictor\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\bella\Documents\GitHub\CountryHappinessPredictor\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\bella\Documents\GitHub\CountryHappinessPredictor\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\bella\Documents\GitHub\CountryHappinessPredictor\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\bella\Documents\GitHub\CountryHappinessPredictor\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empt

In [54]:
# any values left that are still NaN filled with 0s
df_usable = df_usable.fillna(0)

In [55]:
missing_fraction = df_usable.isna().mean()

# count how many have 0 missingness
num_incomplete_columns = (missing_fraction != 0).sum()

print("Number of columns that are incomplete:", num_incomplete_columns)

Number of columns that are incomplete: 0


In [60]:
df_usable["country"] = df["country"]
cols = ["country"] + [c for c in df_usable.columns if c != "country"]
df_usable = df_usable[cols]

In [62]:
df_usable.head(10)

,country,score,year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Adjusted net national income (annual % growth),...,Voice and Accountability: Standard Error,"Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100),Women's share of population ages 15+ living with HIV (%)
0,Switzerland,7.587,2015,100.0,100.0,100.0,100.0,100.0,100.0,3.829646,...,0.152728,10.463604,10.099898,10.268806,86.896059,83.402880,85.024916,406.888604,85.625,26.437007
1,Iceland,7.561,2015,100.0,100.0,100.0,100.0,100.0,100.0,8.662795,...,0.152728,7.496144,16.520870,12.241124,91.548103,81.380837,86.202403,62.673383,96.875,28.673923
2,Denmark,7.527,2015,100.0,100.0,100.0,100.0,100.0,100.0,2.835303,...,0.144910,4.377631,7.459880,6.019697,94.200167,88.443370,91.133237,371.715844,100.000,26.312663
3,Norway,7.522,2015,100.0,100.0,100.0,100.0,100.0,100.0,-0.748130,...,0.142751,4.330660,8.057614,6.281178,95.216584,90.740970,92.874207,143.056507,96.875,33.514336
4,Canada,7.427,2015,100.0,100.0,100.0,100.0,100.0,100.0,-2.396463,...,0.144154,8.982606,11.808995,10.474294,88.454140,81.910413,85.000559,43.563079,97.500,23.485395
5,Finland,7.406,2015,100.0,100.0,100.0,100.0,100.0,100.0,1.971681,...,0.144910,7.813703,13.118529,10.544560,90.671990,81.052217,85.719852,85.544250,97.500,26.988025
6,Netherlands,7.378,2015,100.0,100.0,100.0,100.0,100.0,100.0,3.295244,...,0.144910,10.617352,15.043420,12.993723,87.646676,80.552578,83.837831,92.229661,97.500,18.965681
7,Sweden,7.364,2015,100.0,100.0,100.0,100.0,100.0,100.0,3.561789,...,0.142751,4.404664,9.280488,6.954681,94.010933,85.845618,89.740538,211.241374,100.000,38.433488
8,New Zealand,7.286,2015,100.0,100.0,100.0,100.0,100.0,100.0,4.176460,...,0.142751,12.171621,15.490623,13.934823,83.076957,76.208496,79.428038,18.048017,91.875,17.327112
9,Australia,7.284,2015,100.0,100.0,100.0,100.0,100.0,100.0,0.000000,...,0.144910,6.477619,10.971947,8.899853,90.368070,82.553717,86.156497,146.561429,96.875,12.922281


In [63]:
df_usable.to_parquet("dataset_complete.parquet", index=False)

Checking baseline options

In [2]:
df = pd.read_parquet("dataset_complete.parquet")

In [4]:
keywords = ["income", "gdp", "gni", "wealth", "earnings", "consumption", "expenditure", "remittance"]

income_related_cols = [
    col for col in df.columns
    if any(kw in col.lower() for kw in keywords)
]

In [5]:
df[income_related_cols].head()

,Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2015 US$),Adjusted net national income per capita (current US$),"Adjusted net savings, excluding particulate emission damage (% of GNI)","Adjusted net savings, including particulate emission damage (% of GNI)",Adjusted savings: carbon dioxide damage (% of GNI),Adjusted savings: consumption of fixed capital (% of GNI),...,"Secondary income, other sectors, payments (BoP, current US$)","Services, value added (% of GDP)",Tax revenue (% of GDP),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)","Total alcohol consumption per capita, female (liters of pure alcohol, projected estimates, female 15+ years of age)","Total alcohol consumption per capita, male (liters of pure alcohol, projected estimates, male 15+ years of age)",Total natural resources rents (% of GDP),Trade (% of GDP),Trade in services (% of GDP),"Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)"
0,3.829646,5.495156e+11,5.495156e+11,2.654416,66347.416159,66347.416159,16.821248,16.774353,0.191233,22.687762,...,4.011168e+10,72.031713,9.556179,10.69,4.91,16.68,0.010692,117.138482,35.524788,406.888604
1,8.662795,1.398266e+10,1.398266e+10,7.536472,42267.312470,42267.312470,12.349160,12.332711,0.432984,15.480653,...,3.264994e+08,64.382817,22.689444,7.85,3.62,12.06,0.000300,95.818208,40.978707,62.673383
2,2.835303,2.602923e+11,2.602923e+11,2.111410,45798.029802,45798.029802,17.851647,17.805467,0.375634,16.124136,...,2.189999e+09,66.268124,34.156043,9.61,4.37,15.00,0.521485,105.130362,41.140039,371.715844
3,-0.748130,3.177570e+11,3.177570e+11,-1.730872,61241.287335,61241.287335,19.298012,19.283025,0.341728,17.145534,...,5.803610e+09,55.983372,22.088549,6.83,3.10,10.54,5.162399,70.431857,22.907171,143.056507
4,-2.396463,1.262658e+12,1.262658e+12,-3.135729,35364.119107,35364.119107,5.429334,5.395404,1.267569,17.465922,...,7.804612e+09,67.000998,12.389811,9.92,4.35,15.64,0.332934,66.164902,12.205730,43.563079
